In [88]:
# import lenet
# import importlib
# importlib.reload(lenet)
# from lenet import Lenet
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from torchvision.io import read_image
from torchvision import datasets, transforms
from PIL import Image
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import copy
import random
import torch.nn.functional as F

In [188]:
##
# x=np.random.rand(100,10)
x=[]
y=[]
x1=np.random.rand(100,10)
for i in range(100):
    random_num=random.randint(0,4)
    y.append(random_num)
for j in x1:
    j=j.tolist()
    x.append(j)
df=pd.DataFrame({"x":x,"y":y})
df.iloc[0]


x    [0.7138247490278976, 0.9153898190974633, 0.540...
y                                                    4
Name: 0, dtype: object

In [189]:
df.iloc[0][0]

[0.7138247490278976,
 0.9153898190974633,
 0.5406632839291191,
 0.9776485934074707,
 0.9330016780937085,
 0.5358411086676006,
 0.6953967557609754,
 0.3275152240733403,
 0.3169532114818341,
 0.11639192684906663]

In [190]:
class Dataset(Dataset):
    def __init__(self,df,transforms=None):
        self.df=df
        self.transforms=transforms
    def __len__(self):
        return len(self.df)
    def __getitem__(self, index):
        image=self.df.iloc[index][0]
        label=int(self.df.iloc[index][1])
        if self.transforms:
            image=self.transforms(image)
        return image,torch.tensor(label)
    
image_set=Dataset(df)

In [191]:
data_transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])
dataloader=DataLoader(dataset=image_set,batch_size=1,shuffle=True)

In [192]:
df

,x,y
0,"[0.7138247490278976, 0.9153898190974633, 0.540...",4
1,"[0.897565062806618, 0.2563407856995328, 0.0864...",0
2,"[0.9634150773290283, 0.390328525355095, 0.2811...",3
3,"[0.4343356456589281, 0.8506238855928989, 0.108...",0
4,"[0.6864963679100419, 0.26429870768639685, 0.26...",0
...,...,...
95,"[0.3767964653438509, 0.6713565033929432, 0.753...",3
96,"[0.5042463901838149, 0.6480786873011438, 0.612...",1
97,"[0.8064925446643461, 0.5162774886520386, 0.816...",3
98,"[0.49396475950498087, 0.5741053969208836, 0.49...",3


In [193]:
for i in range(len(df)):
    a=df.iloc[i][0]
    a=torch.tensor(a)
    b=df.iloc[i][1]
    b=torch.tensor(b)
    print(a)
    print(b)

tensor([0.7138, 0.9154, 0.5407, 0.9776, 0.9330, 0.5358, 0.6954, 0.3275, 0.3170,
        0.1164])
tensor(4)
tensor([0.8976, 0.2563, 0.0864, 0.1377, 0.9075, 0.7979, 0.8291, 0.4949, 0.4224,
        0.9604])
tensor(0)
tensor([0.9634, 0.3903, 0.2811, 0.5985, 0.8334, 0.2185, 0.3738, 0.6256, 0.4171,
        0.4965])
tensor(3)
tensor([0.4343, 0.8506, 0.1089, 0.3292, 0.7784, 0.8898, 0.7894, 0.5852, 0.9741,
        0.2964])
tensor(0)
tensor([0.6865, 0.2643, 0.2692, 0.9310, 0.1076, 0.0553, 0.3949, 0.2904, 0.3068,
        0.3803])
tensor(0)
tensor([0.3712, 0.9177, 0.7953, 0.4490, 0.1456, 0.2277, 0.5615, 0.2379, 0.6560,
        0.1110])
tensor(1)
tensor([0.6415, 0.7025, 0.1387, 0.2617, 0.6537, 0.1791, 0.6898, 0.0015, 0.1838,
        0.1032])
tensor(3)
tensor([0.6727, 0.8850, 0.9831, 0.7176, 0.3691, 0.9347, 0.1036, 0.0901, 0.6619,
        0.3652])
tensor(2)
tensor([0.3571, 0.9786, 0.6907, 0.0519, 0.9683, 0.4238, 0.1547, 0.1861, 0.0129,
        0.0685])
tensor(3)
tensor([0.6555, 0.3052, 0.6689, 0.173

In [194]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP,self).__init__()
        self.f1=nn.Linear(10,100)
        self.f2=nn.Linear(100,50)
        self.f3=nn.Linear(50,5)
    def forward(self,x):
        x=self.f1(x)
        x=F.relu(x)
        x=F.relu(self.f2(x))
        x=self.f3(x)

        return x
        


In [231]:
def train(model,df,epoches):
    criterion=torch.nn.CrossEntropyLoss()
    optimizer=torch.optim.SGD(params=model.parameters(),lr=1e-5)
    loss=0
    acc=0.0
    train_acc_all=[]
    train_loss_all=[]
    count=0
    for epoch in range(epoches):
        model.train()
        # for feature, label in dataloader:
        for i in range(len(df)):
            feature=df.iloc[i][0]
            feature=torch.tensor(feature)
            label=df.iloc[i][1]
            label=torch.tensor(label)
            pre=model(feature)
            pre_=torch.unsqueeze(pre,0)
            loss=criterion(pre,label)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            pre_label=torch.argmax(pre,dim=0)

            acc+=torch.sum(pre_label==label)
            loss+=loss.item()*feature.size(0)
            count+=1
        train_acc_all.append(acc.double().item()/count)
        train_loss_all.append(loss/count)
        print("epoch",epoch, "trian_loss",train_loss_all[-1],"train_acc",train_acc_all[-1])
        print("-"*30)
    return train_acc_all,train_loss_all




In [232]:
model=MLP()
dataloader=df
epoches=5
train_acc_all,train_loss_all=train(model,dataloader,epoches)

epoch 0 trian_loss tensor(0.1676, grad_fn=<DivBackward0>) train_acc 0.2
------------------------------
epoch 1 trian_loss tensor(0.0838, grad_fn=<DivBackward0>) train_acc 0.2
------------------------------
epoch 2 trian_loss tensor(0.0559, grad_fn=<DivBackward0>) train_acc 0.2
------------------------------
epoch 3 trian_loss tensor(0.0419, grad_fn=<DivBackward0>) train_acc 0.2
------------------------------
epoch 4 trian_loss tensor(0.0335, grad_fn=<DivBackward0>) train_acc 0.2
------------------------------
